<a href="https://colab.research.google.com/github/stacs-cp/permutation-classes-cp/blob/main/PP2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!source <(curl https://raw.githubusercontent.com/conjure-cp/conjure-notebook/main/installcolab.sh)
%load_ext conjure

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   445  100   445    0     0   3320      0 --:--:-- --:--:-- --:--:--  3320
Cloning into 'conjure-notebook-executable'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 39 (delta 4), reused 33 (delta 4), pack-reused 6
Unpacking objects: 100% (39/39), done.
Checking out files: 100% (34/34), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/conjure-cp/conjure-notebook.git to /tmp/pip-req-build-vf0dicsg
  Running command git clone -q https://github.com/conjure-cp/conjure-notebook.git /tmp/pip-req-build-vf0dicsg
  Created wheel for conjure: filename=conjure-2.3.0-py2.py3-none-any.whl size=9060 sha256=104d39af887e848edd76653baa52103f1fa6fa358d95106760

ModuleNotFoundError: ignored

In [ ]:
length = 4
avoiding = {(sequence(1,3,2), relation((0,0),(1,0),(2,0),(3,0),(1,1),(1,2),(1,3),(0,2),(2,2),(3,2)))}

In [ ]:
%%conjure

given length : int

find perm : sequence (size length, injective) of int(1..length)


UsageError: Cell magic `%%conjure` not found.
